# 中間層の取得
参考URL: https://note.nkmk.me/python-tensorflow-keras-applications-pretrained-models/

In [10]:
import numpy as np
import os
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB7, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models
from IPython.display import display

In [1]:
class CFG:
    debug=True

    models_dir="./models/"

## input_shape:
入力する画像のshapeを指定する, (入力画像数, height, width, チャンネル数)となる。  
1枚だけの処理でも入力画像数を指定しなければいけないので, np.expand_dims()などを用いる。

## include_top:
出力時に画像をクラスに分けるかどうか, imagenetで訓練されたモデルなら1000クラスに分ける。  
Falseにすれば通常だと全結合層以降が除かれる。

## pooling:
モデルの最後にプーリング層をつけるかどうか, avgならGlobalAveragePooling2D層が追加される。

In [15]:
# model=EfficientNetB7(weights="imagenet",include_top=False,pooling="avg")

with open(f"{CFG.models_dir}effnet_tuned/model.json","rb") as f:
    model_json=f.read()

effnet=models.model_from_json(model_json)
effnet.load_weights(f"{CFG.models_dir}effnet_tuned/weight.hdf5")
effnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 9, 9, 2560)        64097687  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 11)                28171     
Total params: 64,125,858
Trainable params: 63,815,131
Non-trainable params: 310,727
_________________________________________________________________


## preprocess_input()
用いるモデルに合わせた前処理を行う

In [36]:
dataset_dir="../input/ranzcr-256x256-dataset/"
dest_dir="../input/efficientnet_output/"

for file_name in tqdm(os.listdir(dataset_dir)):
    img_pil=image.load_img(dataset_dir+file_name) # PIL形式で画像が読み込まれる
    img=image.img_to_array(img_pil)
    img=np.expand_dims(img,axis=0)

    pred=model.predict(preprocess_input(img))
    np.save(dest_dir+os.path.splitext(file_name)[0]+".npy",pred)

100%|██████████| 30083/30083 [13:41:57<00:00,  1.64s/it]
